In [ ]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib.pyplot as plt
import numba
import os
root_folder = os.getcwd()

In [ ]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "restart_nonbiased_Kv=0.off"))### Read The Final Mesh Here

v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "equilibrated.off") ) ### Read The Initial Mesh Here                         

In [ ]:
p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering   
                               "width": 300, "height": 300},return_plot=True) ###Viewing The Mesh

In [ ]:
#Mesh with particl
rp=0.15
rho=0.1*rp
x=np.amax(v_init[:,0])+rp+1*rho
y=0
z=0
fig = plt.figure(figsize =(14, 8))
ax = plt.axes(projection ='3d')
surf=ax.plot_trisurf(v[:,0], v[:,1], v[:,2], triangles = f, edgecolor=[[0,0,0]], linewidth=0.3, alpha=0.0, shade=False)
u, v1 = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
x1 = (rp*(np.cos(u) * np.sin(v1)))+x
y1 = (rp*(np.sin(u) * np.sin(v1)))
z1 = (rp*(np.cos(v1)))
surf=ax.plot_surface(x1, y1, z1)
ax.view_init(90,180)
ax = plt.gca()
ax.set_xlim([-1, 1.5])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
ax.set_proj_type('ortho')
ax.set_box_aspect((xmax-xmin, ymax-ymin,ymax-ymin))
ax.set_zticks([])

In [ ]:
def cal_volumetot(v,f):
    Volumetot = 0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volumetot+=sum
    return Volumetot


In [ ]:
### To chech the initial and final volume
Volume_Final= cal_volumetot(v,f)
Volume_new=cal_volumetot(v_init,f1)
print(Volume_new)
Volume_Final/Volume_new

In [ ]:
Kb=0.01
H0=0
def Force_Bending(v,f):
    npv = igl.per_vertex_normals(v, f)
    K = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    H_mean_signed = H_mean*sign_H
    Lap_H = minv.dot(l.dot(H_mean_signed-H0))
    kn = minv.dot(K)
    first_term = 2*(H_mean_signed-H0)*(H_mean_signed**2 + H0*H_mean_signed -kn)
    totalforce = first_term + Lap_H
    Force_bending = 2*Kb*npv*totalforce[:,None]*area_voronoi[:,None]

    Eb = 2*Kb*(((H_mean_signed-H0)**2))*area_voronoi
    total_EB = np.sum(Eb)
        
    return Force_bending,total_EB

In [ ]:
### To check the bending energy of the vesicle
force,energy=Force_Bending(v,f)
energy

In [ ]:
# with open('logfile_1.txt','r') as f: ###Give the file name here
#     Energy1=[]
#     string='Total Energy' ### Name it as the which energy term you want 
#     for line in f:
#         if string in line:
#             Energy1.append(float(line.split(': ')[1]))
# print(Energy1)

In [ ]:
###To read an Energy contribution from the logfile

with open('logfile.txt','r') as f: ###Give the file name here
    Energy=[]
    string='Total Energy' ### Name it as the which energy term you want 
    for line in f:
        if string in line:
            Energy.append(float(line.split(': ')[1]))
print(Energy)


In [ ]:
number_of_iterations=np.arange(0,len(Energy+Energy1),1)
number_of_iterations=number_of_iterations*100

In [ ]:
###Plotting Energy Contribution with resrpect to number of iterations
import matplotlib as mpl
mpl.rcParams['figure.dpi']=400
plt.plot(number_of_iterations,a,'b.')
csfont={'fontname':'Times New Roman'}
plt.xlabel('Number of Iterations',**csfont)
plt.ylabel('Total Energy of the System',fontsize=12,**csfont)
plt.title('Inside_Particle_u=4.0,rp=0.20')